# Setup

In [1]:
from IPython.display import display, Markdown, Math, HTML, Code
from mogwai.parser import graphml_to_mogwaigraph
from mogwai.core.traversal import MogwaiGraphTraversalSource
from mogwai.core.steps.statics import *
import os

install [gremlinpython](https://pypi.org/project/gremlinpython/)


In [4]:
%pip install gremlinpython

  Using cached aiohttp-3.11.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata (7.7 kB)
  Using cached aenum-3.1.15-py3-none-any.whl.metadata (3.7 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (5.0 kB)
  Using cached propcache-0.2.0-cp310-cp310-macosx_10_9_x86_64.whl.metadata (7.7 kB)
  Using cached yarl-1.17.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata (64 kB)
Using cached aenum-3.1.15-py3-none-any.whl (137 kB)
Using cached aiohttp-3.11.2-cp310-cp310-macosx_10_9_x86_64.whl (466 kB)
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl (14 kB)
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached fr

In [5]:
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import P, T
from gremlin_python.process.graph_traversal import __

In [9]:
# start server
# this can also be done with the https://wiki.bitplan.com/index.php/Gremlin_python project's scripts
import subprocess
gremlin_server_path = '/home/joep/programs/gremlin-server'
gremlin_command = f'{gremlin_server_path}/bin/gremlin-server.sh {gremlin_server_path}/conf/gremlin-server.yaml'
gremlin_server_process = subprocess.Popen(gremlin_command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
display(Markdown("#### ✔ Server gestartet."))

FileNotFoundError: [Errno 2] No such file or directory: '/home/joep/programs/gremlin-server/bin/gremlin-server.sh'

In [60]:
gremlin_server_process.kill()

In [12]:
import socket
try:
   socket.create_connection(('localhost', 8182))
   print("Gremlin Port 8182 is accessible")
except socket.error:
   print("Gremlin Port 8182 is not accessible")

Gremlin Port 8182 is accessible


In [13]:
# MIT SERVER VERBINDEN
# Connect to localhost:8182
graph_gp = Graph()
g_gp = graph_gp.traversal().with_remote(DriverRemoteConnection("ws://127.0.0.1:8182/gremlin", "g", transport_factory=lambda:AiohttpTransport(call_from_event_loop=True)))
display(Markdown("#### ✔ Mit Server Verbunden."))

#### ✔ Mit Server Verbunden.

# Large Airroutes Graph

## Import Graph

In [22]:
graph_mogwai = graphml_to_mogwaigraph("air-routes-latest.graphml", node_label_key="labelV", edge_label_key="labelE", node_name_key="code")
g_m = MogwaiGraphTraversalSource(graph_mogwai)

In [18]:
# local path in mogwai_examples
air_routes_path=os.path.abspath("air-routes-latest.graphml")

In [19]:
# copy to docker if need be 
# skip this step if you run locally
import subprocess
docker_cp_command = f"docker cp {air_routes_path} gremlin-server:/opt/gremlin-server/data/"
subprocess.run(docker_cp_command.split())
# When using Docker, override path for container
air_routes_path = "/opt/gremlin-server/data/air-routes-latest.graphml"

In [20]:
# read the content from the air routes example
g_gp.V().drop().iterate()
g_gp.io(air_routes_path).read().iterate()
v_count=g_gp.V().count().next()
print (f"graph imported from {air_routes_path} has {v_count} vertices")

graph imported from /opt/gremlin-server/data/air-routes-latest.graphml has 3749 vertices


## Query 1

### PyMogwai

In [23]:
g_m.V().has_label('airport').has_name('LAX').as_('start').repeat(
		outE('route').as_('e').inV().filter_(
			select('e').properties('dist').is_(lte(2000))
		).simple_path()
	).times(3).emit().properties("city").dedup().to_list().run()

[{'city': 'Atlanta'},
 {'city': 'Austin'},
 {'city': 'Nashville'},
 {'city': 'Dallas'},
 {'city': 'Houston'},
 {'city': 'Minneapolis'},
 {'city': 'Chicago'},
 {'city': 'Phoenix'},
 {'city': 'Seattle'},
 {'city': 'San Francisco'},
 {'city': 'San Jose'},
 {'city': 'San Diego'},
 {'city': 'Salt Lake City'},
 {'city': 'Las Vegas'},
 {'city': 'Denver'},
 {'city': 'San Antonio'},
 {'city': 'New Orleans'},
 {'city': 'Cedar Rapids'},
 {'city': 'El Paso'},
 {'city': 'Oakland'},
 {'city': 'Tucson'},
 {'city': 'Santa Fe'},
 {'city': 'Detroit'},
 {'city': 'Vancouver'},
 {'city': 'Edmonton'},
 {'city': 'Calgary'},
 {'city': 'Mexico City'},
 {'city': 'Portland'},
 {'city': 'Oaklahoma City'},
 {'city': 'Palm Springs'},
 {'city': 'Memphis'},
 {'city': 'Cincinnati'},
 {'city': 'Indianapolis'},
 {'city': 'Kansas City'},
 {'city': 'St Louis'},
 {'city': 'Albuquerque'},
 {'city': 'Milwaukee'},
 {'city': 'Omaha'},
 {'city': 'Tulsa'},
 {'city': 'Puerto Vallarta'},
 {'city': 'Sacramento'},
 {'city': 'Colorad

### Gremlin-Python

In [24]:
g_gp.V().has_label('airport').has("code",'LAX').as_('start').repeat(
        __.outE('route').as_('e').inV().filter_(
            __.select('e').values('dist').is_(P.lte(2000))
        ).simple_path()
    ).times(3).emit().values('city').dedup().to_list()

['Houston',
 'Fayetteville/Springdale/',
 'San José del Cabo',
 'Eagle',
 'Minneapolis',
 'Zacatecas',
 'Chicago',
 'Hayden',
 'Dallas',
 'Mazatlán',
 'Madison',
 'Morelia',
 'San Francisco',
 'San Jose',
 'San Diego',
 'Vancouver',
 'Salt Lake City',
 'Las Vegas',
 'Denver',
 'Phoenix',
 'Seattle',
 'El Paso',
 'Oakland',
 'Tucson',
 'San Antonio',
 'New Orleans',
 'Cedar Rapids',
 'South Bend',
 'Santa Fe',
 'Aspen',
 'Kelowna',
 'San Luis Obispo',
 'Bakersfield',
 'Arcata/Eureka',
 'Medford',
 'Eugene',
 'Pasco',
 'Monterey',
 'Santa Barbara',
 'Redmond',
 'Santa Maria',
 'Visalia',
 'Edmonton',
 'Santa Rosa',
 'Detroit',
 'Carlsbad',
 'Bellingham',
 'Kingman',
 'Spokane',
 'Mammoth Lakes',
 'Merced',
 'Prescott',
 'Yuma',
 'Bozeman',
 'Provo',
 'Calgary',
 'Hermosillo',
 'Durango',
 'Uruapan',
 'Loreto',
 'Manzanillo',
 'Ixtapa',
 'Oaxaca',
 'Oaklahoma City',
 'Imperial',
 'Portland',
 'Flagstaff',
 'Hailey',
 'Ogden',
 'Jackson',
 'Kalispell',
 'St George',
 'Mexico City',
 'Misso

## Query 2

In [25]:
g_m.V().has_label("airport").within("city", ["Brussels", "Maastricht", "Aachen", "Dusseldorf"]).as_('start').\
        outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest').order(desc=True).by(select('e').properties('dist')).\
        limit(5).select('start', 'dest').to_list().by('desc').run()

[['Dusseldorf International Airport', 'Singapore'],
 ['Brussels Airport', 'Japan'],
 ['Brussels South Charleroi Airport', 'Hong Kong'],
 ['Brussels Airport', 'Hong Kong'],
 ['Brussels Airport', 'China']]

## Pymogwai

In [26]:
from mogwai.core.steps.statics import select
query = g_m.V().has_label("airport").within(["properties","city"], ["Brussels", "Maastricht", "Aachen", "Dusseldorf"]).as_('start')\
        .outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest')\
        .order(desc=True).by(select('e').properties('dist')).\
        limit(5).select('start', 'dest').by('desc').to_list()
res = query.run()
print("Result:", res)

Result: []


## Gremlin-Python

In [27]:
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import P
query = g_gp.V().has_label("airport").has("city", P.within(["Brussels", "Maastricht", "Aachen", "Dusseldorf"])).as_('start')\
            .outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest')\
            .order().by(__.select('e').values('dist'),Order.desc)\
            .limit(5).select('start', 'dest').by("desc").to_list()
print(query)

[{'start': 'Dusseldorf International Airport', 'dest': 'Singapore'}, {'start': 'Brussels Airport', 'dest': 'Japan'}, {'start': 'Brussels South Charleroi Airport', 'dest': 'Hong Kong'}, {'start': 'Brussels Airport', 'dest': 'Hong Kong'}, {'start': 'Brussels Airport', 'dest': 'China'}]


# Query 3

## PyMogwai

In [28]:

from mogwai.core.steps.statics import outE, select, lte, count, Scope
query = g_m.V().has_label('airport').has_property("city", "Maastricht").as_('start').repeat(
            outE('route').as_('e').inV().filter_(
                select('e').properties('dist').is_(lte(2000))
            ).simple_path()
        ).times(3).emit().as_('dest')\
        .path().as_('p').count(Scope.local).order(asc=True).as_('length')\
        .select('dest').by('city').dedup().select('dest', 'length').to_list().by('city')
print("Query:", query.print_query())
res = query.run()
print(res)
#print("Result length", len(res))

AttributeError: 'Traversal' object has no attribute 'has_property'

## Gremlin-Python

In [41]:
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Scope
query = g_gp.V().has_label('airport').has("city", "Maastricht").as_('start').repeat(
				__.outE('route').as_('e').inV().filter_(__.select('e').values('dist').is_(P.lte(2000))
			).simple_path()
        ).times(3).emit().as_('dest')\
        .path().as_('p').count(Scope.local).order().by(Order.asc).as_('length')\
        .select("dest").by("city").dedup().as_("dest").select('dest', 'length').to_list()
print(query)

[{'dest': 'Porto', 'length': 3}, {'dest': 'Faro', 'length': 3}, {'dest': 'Zakynthos', 'length': 3}, {'dest': 'Alghero', 'length': 3}, {'dest': 'Antalya', 'length': 3}, {'dest': 'Trapani', 'length': 3}, {'dest': 'Palma De Mallorca', 'length': 3}, {'dest': 'Girona', 'length': 3}, {'dest': 'Catania', 'length': 3}, {'dest': 'Istanbul', 'length': 3}, {'dest': 'Luqa', 'length': 3}, {'dest': 'Bari', 'length': 3}, {'dest': 'Alicante', 'length': 3}, {'dest': 'Kerkyra Island', 'length': 3}, {'dest': 'Heraklion', 'length': 3}, {'dest': 'Tenerife Island', 'length': 3}, {'dest': 'Kos Island', 'length': 3}, {'dest': 'Beauvais/Tillé', 'length': 5}, {'dest': 'Warsaw', 'length': 5}, {'dest': 'London', 'length': 5}, {'dest': 'Naples', 'length': 5}, {'dest': 'Roma', 'length': 5}, {'dest': 'Bergamo', 'length': 5}, {'dest': 'Brussels', 'length': 5}, {'dest': 'Katowice', 'length': 5}, {'dest': 'Lisbon', 'length': 5}, {'dest': 'Luxembourg', 'length': 5}, {'dest': 'Gran Canaria', 'length': 5}, {'dest': 'Venic

In [42]:
len(query)

880

## Mogwai

In [59]:
g_m.V().has_label("country").has_property("desc", "Germany").out("contains").outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest').select('e').order(desc=True).by("dist").limit(1).select('dest').by('desc').next().run()

'Argentina'

## Gremlin Python

In [45]:
g_gp.V().has_label("country").has("desc", "Germany").out("contains").outE("route").as_("e").inV().in_("contains").has_label("country").as_('dest').select('e').order().by("dist",Order.desc).limit(1).select("dest").by("desc").next()

'Argentina'

# execute queries multiple times

In [ ]:
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Scope
import time
for i in range(10):
    query = g_gp.V().has_label('airport')\
        .has("city", "Maastricht").as_('start')\
            .repeat(
                __.outE('route').as_('e').inV().filter_(__.select('e').values('dist').is_(P.lte(2000))).simple_path()
            )\
                .times(3).emit().as_('dest').path().as_('p')\
                    .count(Scope.local).order().by(Order.asc)\
                        .as_('length').select("dest").by("city").dedup().as_("dest").select('dest', 'length').to_list()

In [ ]:
from mogwai.core.steps.statics import outE, select, lte, count, Scope
for i in range(10):
    query = g_m.V().has_label('airport').has_property("city", "Maastricht").as_('start').repeat(
                outE('route').as_('e').inV().filter_(select('e').properties('dist').is_(lte(2000))).simple_path()
            ).times(3).emit().as_('dest').path().as_('p').count(Scope.local).order(asc=True).as_('length').select('dest').dedup().select('dest', 'length').to_list().by('city')
    print("Query:", query.print_query())
    res = query.run()